In [1]:
# mount google drive 

import os, sys 
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
print(os.getcwd())

/content


In [2]:
!pip install transformers
!pip install tokenizers
!pip install tensorflow_addons

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 890kB 34.4MB/s 
     |████████████████████████████████| 3.2MB 31.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=835fd9b01eeab3118d75e91aa6e51ca6fb249cb90b00651de60c83d289c67d4e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 706kB 4.2MB/s 


In [4]:
# import and necessary fuctions for prediction
import numpy as np
import re
import string
import json
import tensorflow as tf
import tensorflow_addons as tfa
import keras
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel
MAX_LEN = 511
VERBOSE = 2
BATCH_SIZE = 8
tokenizer = BertWordPieceTokenizer("/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/bert-base-multilingual-cased/vocab.txt", lowercase=False)

def normalized_answer(s):
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text)
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

class TFBERTQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBERTQuestionAnswering, self).__init__()
        
        self.encoder = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.start_logit = tf.keras.layers.Dense(num_class, name="start_logit", use_bias=False)
        self.end_logit = tf.keras.layers.Dense(num_class, name="end_logit", use_bias=False)
        self.flatten = tf.keras.layers.Flatten() 
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)
        
    def call(self, inputs):
        input_ids, token_type_ids, attention_mask = inputs
        embedding = self.encoder(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)[0]
        start_logits = self.start_logit(embedding)
        start_logits = self.flatten(start_logits)
        
        end_logits = self.end_logit(embedding)
        end_logits = self.flatten(end_logits)
        
        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)
    
        return start_probs, end_probs

class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = MAX_LEN - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        
        #함정카드
        # Truncate the comments
        if len(input_ids) >  512:
            input_ids = input_ids[0:128] + input_ids[-382:-1]
            attention_mask = attention_mask[0:128] + attention_mask[-382:-1]
            token_type_ids = token_type_ids[0:128] + token_type_ids[-382:-1]
        #####
        
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                answer_text = qa["answers"][0]["text"]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        #item = suqand_example class
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y

path = '/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/'
with open(path+'extract_train_data.json') as f:
    raw_train_data = json.load(f)

with open(path+'extract_dev_data.json') as f:
    raw_eval_data = json.load(f)


train_squad_examples = create_squad_examples(raw_train_data)
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")

eval_squad_examples = create_squad_examples(raw_eval_data)
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

3487 training points created.
435 evaluation points created.


In [ ]:
##model load



loaded_model = TFBERTQuestionAnswering(model_name='/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/bert-base-multilingual-cased/',dir_path='./src/bert_ckpt', num_class=1)
optimizer = tfa.optimizers.RectifiedAdam(lr=1e-5) #tf.keras.optimizers.RAdam(learning_rate=5e-5)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loaded_model.compile(optimizer=optimizer, loss=[loss, loss])
for_batch = [np.array([x_train[0][0]]),np.array([x_train[1][0]]),np.array([x_train[2][0]])]
for_batch_y = [np.array([y_train[0][0]]),np.array([y_train[1][0]])]
history = loaded_model.fit(
    for_batch,
    for_batch_y,
    epochs=1, 
    verbose=VERBOSE,
    batch_size=16,

)

loaded_model.load_weights('/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/6_epochs_extract_radam_weights.h5')


Some layers from the model checkpoint at /content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/bert-base-multilingual-cased/ were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at /content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/bert-base-multilingual-cased/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


1/1 - 56s - loss: 13.7386 - output_1_loss: 7.2176 - output_2_loss: 6.5210


In [ ]:
# load dev_input.txt and make dev_result.txt file which is prediction
with open('/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/question_ex.json') as f:
    question_ex = json.load(f)

eval_data = []
path = '/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/'
with open(path+'data/dev/dev_input.txt') as f:
    for i in f:
        temp = i.split('\t')
        context = temp[0].strip()
        question = temp[1].strip()
        make_data = {'paragraphs':[{'qas':[{'answers':[{'text':None,'answer_start': None,'id':'for inference'}],'question': question}],'context': context}]}
        eval_data.append(make_data)

eval_result = {}
eval_result['version'] = 'for_inference'
eval_result['data'] = eval_data



class SquadExample2:
    def __init__(self, question, context, max_len=511):
        self.question = question
        self.context = context
        self.skip = False
        self.max_len = max_len

    def preprocess(self):
        context = self.context
        question = self.question

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        
        #error handling
        # Truncate the comments
        # if len(input_ids) >  510:
        #     input_ids = input_ids[0:128] + input_ids[-382:-1] #510
        #     attention_mask = attention_mask[0:128] + attention_mask[-382:-1]
        #     token_type_ids = token_type_ids[0:128] + token_type_ids[-382:-1]
        ##### 
        
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples2(raw_data):
    squad_examples = []
    for item in evres:
        context = item["context"][0]
        question = item["question"]     
        squad_eg = SquadExample2(question, context)
        squad_eg.preprocess()
        squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets2(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": []}
    
    for item in squad_examples:
        #item = suqand_example class
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key)) #squad_example.input_ids 
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    
    return x

max_len = 511
inferences = {}
for n in range(len(eval_result['data'])):
    evres = []
    question = eval_result['data'][n]['paragraphs'][0]['qas'][0]['question']
    eval_splits = eval_result['data'][n]['paragraphs'][0]['context'].split('.')
    
    total_txt = []
    temp_txt = []
    permit = 490

    for idx in range(len(eval_splits)):
        permit -= len(eval_splits[idx])
        if permit < 0:
            permit = 490
            total_txt.append([".".join(temp_txt) + '.'])
            temp_txt = []
        
        temp_txt.append(eval_splits[idx])
        if idx == (len(eval_splits) - 1):
            total_txt.append([".".join(temp_txt)])
    evdict = {}

    for ids,str_txt in enumerate(total_txt):
        # if ids == 0:
        evdict['question'] = question
        evdict['context'] = str_txt
        # else:
        #     evdict['context'] = str_txt
        #     evdict['question'] = 'below'
        evres.append(evdict)
        evdict = {}

    evres_squad = create_squad_examples2(evres)
    x_eval = create_inputs_targets2(evres_squad)
    print(f"{len(evres_squad)} evaluation points created.it is from ",n)
    # print("답 : ",normalized_answer(raw_eval_data['data'][n]['paragraphs'][0]['qas'][0]['answers'][0]['text']), "question: ",question)
    pred_start, pred_end = loaded_model.predict(x_eval)

    eval_examples_no_skip = [_ for _ in evres_squad if _.skip == False] # evres_squad or eval_squad_examples
    inference = []
    status = 0
    for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
        squad_eg = eval_examples_no_skip[idx]
        offsets = squad_eg.context_token_to_char

        start = np.argmax(start)

        end = np.argmax(end)
        if start >= len(offsets):
            continue
        pred_char_start = offsets[start][0]
        if end < len(offsets):
            pred_char_end = offsets[end][1]
            pred_ans = squad_eg.context[pred_char_start:pred_char_end]
        else:
            pred_ans = squad_eg.context[pred_char_start:]
        normalized_pred_ans = normalized_answer(pred_ans)
        for qs_ex in question_ex[question]:
            if normalized_answer(qs_ex.strip()) != '':
                if (normalized_answer(qs_ex.strip()) in normalized_pred_ans) or (normalized_pred_ans in normalized_answer(qs_ex.strip())):
                    if normalized_pred_ans != '':
                        inference.append(normalized_pred_ans)
                        status = 1
                else:
                    if (normalized_pred_ans[:int(len(normalized_pred_ans) / 3)] in normalized_answer(qs_ex.strip())) or (normalized_pred_ans[1 * int(len(normalized_pred_ans) / 4):] in normalized_answer(qs_ex.strip())):
                        if normalized_pred_ans != '':
                            inference.append(normalized_pred_ans)
                            status = 1
        inference = np.unique(inference).tolist()
    if status == 1:
        if (inference != ['']):
            inferences[n] = inference
    else:
        temp_li = []
        for qex in question_ex[question]:
            if normalized_pred_ans[:5] in qex:
                temp_li.append(qex)
        len_max = len(temp_li)
        if len_max == 0:
            inferences[n] = [normalized_answer(question_ex[question][np.random.randint(0,len(question_ex[question]))])]
        else:
            inferences[n] = [normalized_answer(temp_li[np.random.randint(0,len_max)])]
#inferences
ban_list = ['a 환자분 입원하러 오셨나요 b 네','a 입원하러 오셨나요 b 네 오후 4시에 병동으로 올라오라고 했어요','a 네 되셨습니다','a 안녕하세요 혹시 오늘 어떤 것 때문에 병원에 오시게 되셨을까요 b 아 네 안녕하세요 다름이 아니라 대변에서 피가나오는 증상이랑 설사 증상이 있어서 방문하게 되었습니다 b 일주일 전쯤에 상한 음식을 먹고 계속 그러네요','a 안녕하세요 혹시 오늘 어떤 것 때문에 병원에 오시게 되셨을까요 b 숨쉬기가 힘들어요 a 언제부터 그러셨나요 b 몇 일 전부터 기침이 계속 나오더니 삼십분전부터 숨쉬기가 힘들어요','a 안녕하세요 혹시 오늘 어떤 것 때문에 병원에 오시게 되셨을까요 b 숨쉬기가 힘들어요','a 기형이 있으신가요 b 아니요 없습니다','a 흡연을 하시나요 b 네 그렇습니다','a 발진이 있으신가요 b 아니요 없습니다','a 흡연을 하시나요 b 네 그렇습니다 a 흡연량이 얼마나 되나요 b 일주일에 한 갑 피웁니다 a 흡연기간이 얼마나 되시나요 b 30년 정도 되었습니다 a 음주를 하시나요 b 네 그렇습니다 a 음주량과 횟수 종류가 어떻게 되시나요 b 일주일에 23번 마시고 한 번에 소주 2병 정도 마십니다 a 음주기간이 얼마나 되시나요 b 30년 조금 넘었습니다 a 어디를 통해서 입원을 하셨나요 b 이 병원 외래를 방문 후 입원하게 되었습니다','a 최근 삼 개월 이내에 입원치료를 받으신 적이 있나요 b 아니요 없습니다 a 흡연을 하시나요 b 네 그렇습니다 a 흡연량이 얼마나 되나요 b 일주일에 한 갑 피웁니다 a 흡연기간이 얼마나 되시나요 b 30년 정도 되었습니다 a 음주를 하시나요 b 네 그렇습니다 a 음주량과 횟수 종류가 어떻게 되시나요 b 일주일에 23번 마시고 한 번에 소주 2병 정도 마십니다 a 음주기간이 얼마나 되시나요 b 30년 조금 넘었습니다 a 어디를 통해서 입원을 하셨나요 b 이 병원 외래를 방문 후 입원하게 되었습니다','a 음식이나 약에 알레르기 반응이 있으신가요 b 아니요 없습니다','a 수술이나 입원치료를 받으신 적이 있으신가요 b 아니요 없습니다','a 수혈을 하신 적이 있으신가요 b 아니요 없습니다','a 현재 통증이 있으신가요 b 네 있습니다','a 최근 삼 개월 이내에 입원치료를 받으신 적이 있나요 b 아니요 없습니다 a 흡연을 하시나요 b 네 그렇습니다 a 흡연량이 얼마나 되나요 b 일주일에 한 갑 피웁니다 a 흡연기간이 얼마나 되시나요 b 30년 정도 되었습니다 a 음주를 하시나요 b 네 그렇습니다 a 음주량과 횟수 종류가 어떻게 되시나요 b 일주일에 23번 마시고 한 번에 소주 2병 정도 마십니다 a 음주기간이 얼마나 되시나요 b 30년 조금 넘었습니다 a 어디를 통해서 입원을 하셨나요 b 이 병원 외래를 방문 후 입원하게 되었습니다']
for i in inferences.keys():
    if len(inferences[i]) > 1:
        not_ban = []
        for inf in inferences[i]:
            if inf not in ban_list:
                not_ban.append(inf)
        inferences[i] = not_ban
        
        if len(inferences[i]) > 1:
            inferences[i] = [inferences[i][0]]
            
with open('/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/result/dev_result.txt','w',encoding='utf-8') as f:
    for ke in inferences.keys():
        if inferences[ke] == []:
            f.write('A: ~ 인가요? B: 네. 아니요')
            f.write('\n')
        else:
            f.write(inferences[ke][0])
            f.write('\n')

4 evaluation points created.it is from  0
4 evaluation points created.it is from  1
4 evaluation points created.it is from  2
this is random
4 evaluation points created.it is from  3
4 evaluation points created.it is from  4
this is random
4 evaluation points created.it is from  5
4 evaluation points created.it is from  6
4 evaluation points created.it is from  7
4 evaluation points created.it is from  8
4 evaluation points created.it is from  9
2 evaluation points created.it is from  10
2 evaluation points created.it is from  11
2 evaluation points created.it is from  12
2 evaluation points created.it is from  13
2 evaluation points created.it is from  14
6 evaluation points created.it is from  15
6 evaluation points created.it is from  16
6 evaluation points created.it is from  17
6 evaluation points created.it is from  18
6 evaluation points created.it is from  19
this is random
6 evaluation points created.it is from  20
this is random
6 evaluation points created.it is from  21
6 ev

In [ ]:
# load test_input.txt and make test_result.txt file
import numpy as np

with open('/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/src/question_ex.json') as f:
    question_ex = json.load(f)


eval_data = []
path = '/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/'
with open(path+'data/test/test_input.txt') as f:
    for i in f:
        temp = i.split('\t')
        context = temp[0].strip()
        question = temp[1].strip()
        make_data = {'paragraphs':[{'qas':[{'answers':[{'text':None,'answer_start': None,'id':'for inference'}],'question': question}],'context': context}]}
        eval_data.append(make_data)

eval_result = {}
eval_result['version'] = 'for_inference'
eval_result['data'] = eval_data


class SquadExample2:
    def __init__(self, question, context, max_len=511):
        self.question = question
        self.context = context
        self.skip = False
        self.max_len = max_len

    def preprocess(self):
        context = self.context
        question = self.question

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return
        
        #error handling
        # Truncate the comments
        # if len(input_ids) >  510:
        #     input_ids = input_ids[0:128] + input_ids[-382:-1] #510
        #     attention_mask = attention_mask[0:128] + attention_mask[-382:-1]
        #     token_type_ids = token_type_ids[0:128] + token_type_ids[-382:-1]
        ##### 
        
        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.context_token_to_char = tokenized_context.offsets


def create_squad_examples2(raw_data):
    squad_examples = []
    for item in evres:
        context = item["context"][0]
        question = item["question"]     
        squad_eg = SquadExample2(question, context)
        squad_eg.preprocess()
        squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets2(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": []}
    
    for item in squad_examples:
        #item = suqand_example class
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key)) #squad_example.input_ids 
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    
    return x

max_len = 511
inferences = {}
for n in range(len(eval_result['data'])):
    evres = []
    question = eval_result['data'][n]['paragraphs'][0]['qas'][0]['question']
    eval_splits = eval_result['data'][n]['paragraphs'][0]['context'].split('.')
    
    total_txt = []
    temp_txt = []
    permit = 490

    for idx in range(len(eval_splits)):
        permit -= len(eval_splits[idx])
        if permit < 0:
            permit = 490
            total_txt.append([".".join(temp_txt) + '.'])
            temp_txt = []
        
        temp_txt.append(eval_splits[idx])
        if idx == (len(eval_splits) - 1):
            total_txt.append([".".join(temp_txt)])
    evdict = {}

    for ids,str_txt in enumerate(total_txt):

        evdict['question'] = question
        evdict['context'] = str_txt

        evres.append(evdict)
        evdict = {}

    evres_squad = create_squad_examples2(evres)
    x_eval = create_inputs_targets2(evres_squad)
    print(f"{len(evres_squad)} evaluation points created.it is from ",n)

    pred_start, pred_end = loaded_model.predict(x_eval)

    eval_examples_no_skip = [_ for _ in evres_squad if _.skip == False] # evres_squad or eval_squad_examples
    inference = []
    status = 0
    for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
        squad_eg = eval_examples_no_skip[idx]
        offsets = squad_eg.context_token_to_char

        start = np.argmax(start)

        end = np.argmax(end)
        if start >= len(offsets):
            continue
        pred_char_start = offsets[start][0]
        if end < len(offsets):
            pred_char_end = offsets[end][1]
            pred_ans = squad_eg.context[pred_char_start:pred_char_end]
        else:
            pred_ans = squad_eg.context[pred_char_start:]
        normalized_pred_ans = normalized_answer(pred_ans)
        for qs_ex in question_ex[question]:
            if normalized_answer(qs_ex.strip()) != '':
                if (normalized_answer(qs_ex.strip()) in normalized_pred_ans) or (normalized_pred_ans in normalized_answer(qs_ex.strip())):
                    if normalized_pred_ans != '':
                        inference.append(normalized_pred_ans)
                        status = 1
                else:
                    if (normalized_pred_ans[:int(len(normalized_pred_ans) / 3)] in normalized_answer(qs_ex.strip())) or (normalized_pred_ans[1 * int(len(normalized_pred_ans) / 4):] in normalized_answer(qs_ex.strip())):
                        if normalized_pred_ans != '':
                            inference.append(normalized_pred_ans)
                            status = 1
        inference = np.unique(inference).tolist()
    if status == 1:
        if (inference != ['']):
            inferences[n] = inference
    else:
        temp_li = []
        for qex in question_ex[question]:
            if normalized_pred_ans[:5] in qex:
                temp_li.append(qex)
        len_max = len(temp_li)
        if len_max == 0:
            inferences[n] = [normalized_answer(question_ex[question][np.random.randint(0,len(question_ex[question]))])]
        else:
            inferences[n] = [normalized_answer(temp_li[np.random.randint(0,len_max)])]
#inferences
ban_list = ['a 환자분 입원하러 오셨나요 b 네','a 입원하러 오셨나요 b 네 오후 4시에 병동으로 올라오라고 했어요','a 네 되셨습니다','a 안녕하세요 혹시 오늘 어떤 것 때문에 병원에 오시게 되셨을까요 b 아 네 안녕하세요 다름이 아니라 대변에서 피가나오는 증상이랑 설사 증상이 있어서 방문하게 되었습니다 b 일주일 전쯤에 상한 음식을 먹고 계속 그러네요','a 안녕하세요 혹시 오늘 어떤 것 때문에 병원에 오시게 되셨을까요 b 숨쉬기가 힘들어요 a 언제부터 그러셨나요 b 몇 일 전부터 기침이 계속 나오더니 삼십분전부터 숨쉬기가 힘들어요','a 안녕하세요 혹시 오늘 어떤 것 때문에 병원에 오시게 되셨을까요 b 숨쉬기가 힘들어요','a 기형이 있으신가요 b 아니요 없습니다','a 흡연을 하시나요 b 네 그렇습니다','a 발진이 있으신가요 b 아니요 없습니다','a 흡연을 하시나요 b 네 그렇습니다 a 흡연량이 얼마나 되나요 b 일주일에 한 갑 피웁니다 a 흡연기간이 얼마나 되시나요 b 30년 정도 되었습니다 a 음주를 하시나요 b 네 그렇습니다 a 음주량과 횟수 종류가 어떻게 되시나요 b 일주일에 23번 마시고 한 번에 소주 2병 정도 마십니다 a 음주기간이 얼마나 되시나요 b 30년 조금 넘었습니다 a 어디를 통해서 입원을 하셨나요 b 이 병원 외래를 방문 후 입원하게 되었습니다','a 최근 삼 개월 이내에 입원치료를 받으신 적이 있나요 b 아니요 없습니다 a 흡연을 하시나요 b 네 그렇습니다 a 흡연량이 얼마나 되나요 b 일주일에 한 갑 피웁니다 a 흡연기간이 얼마나 되시나요 b 30년 정도 되었습니다 a 음주를 하시나요 b 네 그렇습니다 a 음주량과 횟수 종류가 어떻게 되시나요 b 일주일에 23번 마시고 한 번에 소주 2병 정도 마십니다 a 음주기간이 얼마나 되시나요 b 30년 조금 넘었습니다 a 어디를 통해서 입원을 하셨나요 b 이 병원 외래를 방문 후 입원하게 되었습니다','a 음식이나 약에 알레르기 반응이 있으신가요 b 아니요 없습니다','a 수술이나 입원치료를 받으신 적이 있으신가요 b 아니요 없습니다','a 수혈을 하신 적이 있으신가요 b 아니요 없습니다','a 현재 통증이 있으신가요 b 네 있습니다','a 최근 삼 개월 이내에 입원치료를 받으신 적이 있나요 b 아니요 없습니다 a 흡연을 하시나요 b 네 그렇습니다 a 흡연량이 얼마나 되나요 b 일주일에 한 갑 피웁니다 a 흡연기간이 얼마나 되시나요 b 30년 정도 되었습니다 a 음주를 하시나요 b 네 그렇습니다 a 음주량과 횟수 종류가 어떻게 되시나요 b 일주일에 23번 마시고 한 번에 소주 2병 정도 마십니다 a 음주기간이 얼마나 되시나요 b 30년 조금 넘었습니다 a 어디를 통해서 입원을 하셨나요 b 이 병원 외래를 방문 후 입원하게 되었습니다']
for i in inferences.keys():
    if len(inferences[i]) > 1:
        not_ban = []
        for inf in inferences[i]:
            if inf not in ban_list:
                not_ban.append(inf)
        inferences[i] = not_ban
        
        if len(inferences[i]) > 1:
            inferences[i] = [inferences[i][0]]
            
with open('/content/gdrive/Shareddrives/슈퍼학부생 CREW/hackathon_2-2/result/test_result.txt','w',encoding='utf-8') as f:
    for ke in inferences.keys():
        if inferences[ke] == []:
            f.write('A: ~ 인가요? B: 네. 아니요')
            f.write('\n')
        else:
            f.write(inferences[ke][0])
            f.write('\n')

4 evaluation points created.it is from  0


NameError: ignored